**Лабораторная работа №6. Сегментация текста**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
from PIL import Image, ImageDraw
from PIL.ImageOps import invert

Алфавит - Тибетский алфавит

Исходная фраза - ངས་ཁྱོད་ལ་སྙིང་གི་ཚིགས་པ་ལྟ་བུར་འདོད་དོ།

In [ ]:
def first_nonzero(arr):
    try:
        return np.nonzero(arr)[0][0]
    except IndexError:
        return -1

def last_nonzero(arr):
    try:
        return np.shape(arr)[0] - np.nonzero(np.flip(arr))[0][0] - 1
    except IndexError:
        return -1

def profile_y(input_array):
    output_array2 = input_array
    return {'y_axis' : np.arange(0, output_array2.shape[0]).astype(int), # разметка оси y, 0 тк первое значение размера это колво строк
            'values' : np.sum(output_array2, axis=1) # проекция на ось y
            }

def profile_x(input_array):
    output_array1 = input_array
    return {'values' : np.sum(output_array1, axis=0), # проекция на ось x
            'x_axis' : np.arange(0, output_array1.shape[1]).astype(int) # разметка оси x
            }

def my_X_diagram(input_array):

    height, width = input_array.shape[:2]
    p = profile_x(input_array)
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for x in range(width):
        for y in range(height):
            if y + 1 <= p['values'][x]:
            # Заполняем пиксели черным цветом до уровня проекции
                output_array[y][x] = 0
            else:
                output_array[y][x] = 255  # Остальные пиксели белым цветом

    new_image = Image.fromarray(output_array.astype(np.uint8))
    new_image.save('profile_x.png')


def my_Y_diagram(input_array):
    height, width = input_array.shape[:2]
    p = profile_y(input_array)
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for x in range(width):
        for y in range(height):
            if x+1 <= p['values'][y]:
                output_array[y][x] = 0
            else:
                output_array[y][x] = 255

    new_image = Image.fromarray(output_array.astype(np.uint8))
    new_image.save('profile_y.png')

def monochrome(input_array):
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for y in range(height):
        for x in range(width):
            output_array[y][x] = np.mean(input_array[y][x])

    return output_array

def binarization(input_array):
    height, width = input_array.shape[:2]
    input_array1 = monochrome(input_array)
    output_array = np.zeros((height, width), int)

    for i in range(height):
        for j in range(width):
            if input_array1[i][j] <= 200:
                output_array[i][j] = 1
            else:
                output_array[i][j] = 0

    return output_array

def edit_img(img):
    img_array = np.array(img)

    img_array_01 = binarization(img_array)
    img_array_cut = img_array_01[first_nonzero( profile_y(img_array_01)['values'] ) : last_nonzero( profile_y(img_array_01)['values'] ),
                                 first_nonzero( profile_x(img_array_01)['values'] ) : last_nonzero( profile_x(img_array_01)['values'] )]

    return img_array_cut



In [ ]:
def crop_blank_regions(input_array):
    height, width = input_array.shape[:2]

    top = 0
    bottom = height - 1
    while top < height:
        row = input_array[top]
        if np.any(row != 255):
            break
        top += 1
    while bottom >= 0:
        row = input_array[bottom]
        if np.any(row != 255):
            break
        bottom -= 1

    left = 0
    right = width - 1
    while left < width:
        col = input_array[top:bottom+1, left]
        if np.any(col.flatten() != 255):
            break
        left += 1
    while right >= 0:
        col = input_array[top:bottom+1, right]
        if np.any(col.flatten() != 255):
            break
        right -= 1

    cropped = input_array[top:bottom+1, left:right+1]

    new_image = Image.fromarray(cropped)
    new_image.save('cropped.png')

def segmentation(input_array):
    p = profile_x(input_array)
    height, width = input_array.shape[:2]

    borders = {'left': [], 'right': [], 'up': [], 'down': []}

    for i in range(p['x_axis'].shape[0] - 1):
        if p['values'][i] == 0 and p['values'][i + 1] > 0:
            borders['left'].append(i)
        elif p['values'][i] > 0 and p['values'][i + 1] == 0:
            borders['right'].append(i)

    if borders['left'] and borders['left'][0] > borders['right'][0]:
        borders['left'].insert(0, 0)
    if borders['right'] and borders['right'][-1] < width - 1:
        borders['right'].append(width - 1)

    borders['left'].sort()
    borders['right'].sort()

    for left, right in zip(borders['left'], borders['right']):
        if left < right:
            segment = input_array[:, left:right]
            py = profile_y(segment)
            up = first_nonzero(py['values'])
            down = last_nonzero(py['values'])
            borders['up'].append(up)
            borders['down'].append(down)

    return borders

def draw_brders(input_image, list_borders):
    img_array = np.array(input_image)
    img = Image.fromarray(img_array)
    draw = ImageDraw.Draw(img)

    for k in range(len(list_borders['left'])):
        x1, x2 = list_borders['left'][k], list_borders['right'][k]
        y1, y2 = list_borders['up'][k], list_borders['down'][k]
        draw.rectangle([(x1, y1), (x2, y2)], outline='blue', width=2)

    img.save('segmentation.png')


In [ ]:
def main():

    input_image = Image.open('text1.bmp')
    input_array = np.array(input_image)

    cropped_image = crop_blank_regions(input_array)

    input_image = Image.open('cropped.png')
    input_array = np.array(input_image)

    segment = edit_img(input_array)

    my_X_diagram(segment)
    my_Y_diagram(segment)

    list = segmentation(segment)
    draw_brders(input_image , list)


if __name__ == '__main__':
    main()